# [Part 4] - Encrypted Federated Learning using PySyft

In this notebook we're using the structure of the dataset [A Pressure Map Data set for In-bed Posture Classification](https://physionet.org/content/pmd/1.0.0/) to simulate a situation in which each subject has collected their data with the use of a mobile app. Our intention is to train a global model making use of each individual's data, without having the need to see or move the data out from their devices. For this, we're going to use Federated Learning, which allows us to train individual models inside each device, and average them together to create a global model. This way, no data is revealed to any third party.

We have a folder called __dataset__, which contains two experiments (downloaded in part 1). In these project we're only using 'Experiment I'.

## Import Libraries

In [1]:
import numpy as np
import os
from itertools import islice
import torch as th
import syft as sy
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt

W0820 13:26:00.819633 4394591680 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0820 13:26:00.834891 4394591680 deprecation_wrapper.py:119] From /anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
%matplotlib inline

## Building our subjects database (revisited)

We want to simulate a situation where all data is stored inside each subject's device. For this, we're first going to retrieve the data and store it in a dictionary. The structure will be `{"SubjectName": (dataTensor, labelsTensor)}`

In [3]:
import torch as th

path_exp_1 = "dataset/experiment-i"
positions = ["justAPlaceholder","supine", "right", "left", "right", "right", "left", "left", "supine", "supine", "supine", "supine", "supine", "right", "left", "supine", "supine", "supine"]

subjects_dict = {}

for _, dirs, _ in os.walk(path_exp_1):
  for directory in dirs:
    # each directory is a subject
    subject = directory
    data = None
    labels = None
    
    for _, _, files in os.walk(os.path.join(path_exp_1, directory)):
      for file in files:
        file_path = os.path.join(path_exp_1, directory, file)
        with open(file_path, 'r') as f:
          # Start from second recording, as the first two are corrupted
          for line in f.read().splitlines()[2:]:
            def token_position(x):
              return {
                'supine': 0,
                'left': 1,
                'right': 2,
                'left_fetus': 1,
                'right_fetus': 2
              }[x]
            
            
            raw_data = np.fromstring(line, dtype=float, sep='\t').reshape(1,64,32)
            file_data = np.round(raw_data*255/1000).astype(np.uint8) # Change the range from [0-1000] to [0-255]. This allows us to use tranforms later.
            file_label = token_position(positions[int(file[:-4])]) # Turn the file index into position list, and turn position list into reduced indices.
            file_label = np.array([file_label])
            
            if data is None:
              data = file_data
            else:
              data = np.concatenate((data, file_data), axis=0)

            if labels is None:
              labels = file_label
            else:
              labels = np.concatenate((labels, file_label), axis=0)
              
    subjects_dict[subject] = (th.from_numpy(data), th.from_numpy(labels))

## Create the workers

Now that we have our data partitioned, let's create a worker for each subject. We'll also create a secure worker, which will provide encryption mechanisms

In [4]:
%%capture
# We don't want the download process to fill our screens
!pip install syft
import syft as sy

hook = sy.TorchHook(th)

workers = [sy.VirtualWorker(id=key, hook=hook) for key in subjects_dict.keys()]

secure_worker = sy.VirtualWorker(id="secure_worker", hook=hook)

## Create the datasets and dataloaders

Let's create a dataset and their respective trainloader and testloader for each subject

In [5]:
import copy

# Let's create a copy of the data dict. This way the original dict is untouched.
subj_dict = copy.deepcopy(subjects_dict)
datasets = {}
trainloaders = {}
testloaders = {}
train_percent = 0.8

for worker in workers:
  # Create the Dataset
  datasets[worker.id] = sy.BaseDataset(*subj_dict[worker.id])
  
  train_size = int(train_percent * len(datasets[worker.id]))
  test_size = len(datasets[worker.id]) - train_size
  
  # Split the dataset for the dataloaders
  train_dataset, test_dataset = th.utils.data.random_split(datasets[worker.id], [train_size, test_size])
  
  trainloaders[worker.id] = th.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
  testloaders[worker.id] = th.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)
  
  # Send the dataset to the worker
  datasets[worker.id] = datasets[worker.id].send(worker)
  

## Define the model

This model will be copied and sent to every subject, trained locally, and then we obtain the average through the use of Additive Secret Sharing.

In [6]:
import torch
from torch import nn
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        #Input channels = 1, output channels = 6
        self.conv1 = torch.nn.Conv2d(1, 6, kernel_size=3, stride=1, padding=1)
        self.conv2 = torch.nn.Conv2d(6, 18, kernel_size=3, stride=1, padding=1)
        
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        
        # Inputs to hidden layer linear transformation
        self.h1 = nn.Linear(18 * 16 * 8, 392)
        self.h2 = nn.Linear(392, 98)
        self.h3 = nn.Linear(98, 3)
        
        # Define sigmoid activation and softmax output 
        self.relu = nn.ReLU()
        self.logsoftmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        x = x.float()
        # Add a "channel dimension"
        x = x.unsqueeze(1)
        
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        
        x = x.view(x.shape[0], -1)

        x = self.relu(self.h1(x))
        x = self.relu(self.h2(x))
        x = self.logsoftmax(self.h3(x))
        
        return x
      
net = Network()
net

Network(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(6, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (h1): Linear(in_features=2304, out_features=392, bias=True)
  (h2): Linear(in_features=392, out_features=98, bias=True)
  (h3): Linear(in_features=98, out_features=3, bias=True)
  (relu): ReLU()
  (logsoftmax): LogSoftmax()
)

## Train on each subject

Now we have a model and each subject containing their private datasets. For training we're going to make a copy of the model to each individual, and train remotely using their dataset. It's important to remember that since each dataset is composed of a single individual's examples, it's highly probable that the training will result in overfitting. 

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu" # TODO: research if cuda is supported on federated learning
net.to(device)

models = {}

for worker in workers:
  models[worker.id] = net.copy().send(worker)

In [8]:
from torch import optim

for worker in workers:
  
  model = models[worker.id]
  trainloader = trainloaders[worker.id]
  testloader = testloaders[worker.id]
  
  criterion = nn.NLLLoss()
  optimizer = optim.Adam(model.parameters(), lr = 0.00003)
  
  epochs = 3
  running_loss = 0
  train_losses, test_losses = [], []
  
  for epoch in range(epochs):
    for inputs, labels in trainloader:
      
      inputs, labels = inputs.to(device), labels.to(device)
      
      optimizer.zero_grad()
      
      logps = model.forward(inputs)
      loss = criterion(logps, labels)
      loss.backward()
      optimizer.step()
      
      running_loss += loss
      
    else:
      
      test_loss = 0
      accuracy = 0
      model.eval()
    
      with torch.no_grad():
        for inputs, labels in testloader:
          inputs, labels = inputs.to(device), labels.to(device)
          logps = model.forward(inputs)
          test_loss += criterion(logps, labels)
        
          ps = torch.exp(logps)
          top_p, top_class = ps.topk(1, dim=1)
          equals = top_class == labels.view(*top_class.shape)
          accuracy += torch.mean(equals.float())
    
      train_losses.append(running_loss/len(trainloader))
      test_losses.append(test_loss/len(testloader))
        
      print(f"Subject {worker.id}.. "
            f"Epoch {epoch+1}/{epochs}.. "
            f"Train loss: {running_loss.get()/len(trainloader):.3f}.. "
            f"Test loss: {test_loss.get()/len(testloader):.3f}.. "
            f"Test accuracy: {accuracy.get()/len(testloader):.3f}")
      running_loss = 0
      model.train()
      

Subject S5.. Epoch 1/3.. Train loss: 0.243.. Test loss: 0.028.. Test accuracy: 1.000
Subject S5.. Epoch 2/3.. Train loss: 0.015.. Test loss: 0.008.. Test accuracy: 1.000
Subject S5.. Epoch 3/3.. Train loss: 0.006.. Test loss: 0.004.. Test accuracy: 1.000
Subject S2.. Epoch 1/3.. Train loss: 0.500.. Test loss: 0.150.. Test accuracy: 1.000
Subject S2.. Epoch 2/3.. Train loss: 0.092.. Test loss: 0.049.. Test accuracy: 1.000
Subject S2.. Epoch 3/3.. Train loss: 0.034.. Test loss: 0.023.. Test accuracy: 1.000
Subject S3.. Epoch 1/3.. Train loss: 0.435.. Test loss: 0.077.. Test accuracy: 1.000
Subject S3.. Epoch 2/3.. Train loss: 0.036.. Test loss: 0.016.. Test accuracy: 1.000
Subject S3.. Epoch 3/3.. Train loss: 0.011.. Test loss: 0.007.. Test accuracy: 1.000
Subject S4.. Epoch 1/3.. Train loss: 0.309.. Test loss: 0.057.. Test accuracy: 1.000
Subject S4.. Epoch 2/3.. Train loss: 0.026.. Test loss: 0.014.. Test accuracy: 1.000
Subject S4.. Epoch 3/3.. Train loss: 0.009.. Test loss: 0.007.. T

# Combine the models into a global model and update local models

We want to take advantage of what each model learned so that we get a better, more generalized model for all our subjects. For this, we're going to average all of the models' parameters and use this average to build our global model. This average should result in a model with good accuracy for all subjects. Once we build our global model, we can send it back to each subject so that all devices take advantage of our generalized model.

## Share the parameter's of each subject's model

We use Additive Secret Sharing to allow operations to be performed between models and tensors without sacrificing privacy. This way we can safely combine all parameters without compromising any data.

In [9]:
# There seems to be a bug with fix_prec() on model parameters. (https://github.com/OpenMined/PySyft/issues/2490)
# For this reason we have to do fix_prec().share() directly inside the pointers

for worker in workers:
  model = models[worker.id]
  for p in model.parameters():
    # This is equivalent to model.fix_precision().share(*workers, crypto_provider=secure_worker)
    p.data = p.data.fix_precision().share(*workers, crypto_provider=secure_worker)
    
  models[worker.id] = model
    

In [10]:
# Sanity Check

param = list(models['S1'].parameters())[0]

# This should be (Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
param.location._objects[param.id_at_location]

(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:39500411724 -> S5:26126858823]
	-> [PointerTensor | me:14127617727 -> S2:47416863968]
	-> [PointerTensor | me:67162268495 -> S3:68678061802]
	-> [PointerTensor | me:88426740161 -> S4:54765261537]
	-> [PointerTensor | me:40428048369 -> S10:97459749673]
	-> [PointerTensor | me:76059908807 -> S11:68114333795]
	-> [PointerTensor | me:7485253650 -> S8:14829575712]
	-> [PointerTensor | me:33759476462 -> S1:29512608699]
	-> [PointerTensor | me:54708755841 -> S6:26236504876]
	-> [PointerTensor | me:57632845670 -> S7:80817756787]
	-> [PointerTensor | me:40146488240 -> S9:4586128961]
	-> [PointerTensor | me:40712918845 -> S13:89769970463]
	-> [PointerTensor | me:27077482814 -> S12:46688925286]
	*crypto provider: secure_worker*

## Move the encrypted models to our secure worker

Note that the data is still encrypted and shared among all devices. This is only so that all the aggregation is executed by the secure_worker.

In [11]:
for m_id in models:
  models[m_id].move(secure_worker)

## Combine the parameters

Here we're collecting each model's parameters and averaging them together. Once the average is obtained, we assign it to our global model.

In [12]:
# Iterate over every layer of the global model
for i, layer in enumerate(list(net.children())):
  
  # Check if current layer has weights and biases
  if len(list(layer.parameters())) != 2:
    continue
  
  # Set variables to store the aggregation
  weight = None
  bias = None
  
  # Iterate over every model
  for m_id in models:
    model = models[m_id]
    m_layer = list(model.children())[i]
    
    # Aggregate current layer's weight
    if weight is None:
      weight = m_layer.weight
    else:
      weight += m_layer.weight
    
    # Aggregate current layer's bias
    if bias is None:
      bias = m_layer.bias
    else:
      bias += m_layer.bias
      
      
  # Assign the parameters to our global model
  with th.no_grad():
    
    # Would be nice to do the mean inside secure_worker,
    # but float_prec() is not currently supported on pointer tensors 
    # https://github.com/OpenMined/PySyft/pull/2443
    
    layer.weight.set_(weight.get().get().float_prec()/len(models))
    layer.bias.set_(bias.get().get().float_prec()/len(models))

## Update all local models with out global model

Our global model represents a more generalized model. Now we can replace each subject's models with this one. This way, we're obtaining the benefit of using multiple subject's data while retaining privacy.

In [13]:
for worker in workers:
  models[worker.id] = net.copy().send(worker)

## Check the results

This model should have a good accuracy on every subject. Let's test this hypothesis by running the model locally on our remote testloaders

In [14]:
for worker in workers:
  
  model = models[worker.id]
  testloader = testloaders[worker.id]
  
  criterion = nn.NLLLoss()
  
  test_loss = 0
  accuracy = 0
  model.eval()
    
  with torch.no_grad():
    for inputs, labels in testloader:
      inputs, labels = inputs.to(device), labels.to(device)
      logps = model.forward(inputs)
      test_loss += criterion(logps, labels)
        
      ps = torch.exp(logps)
      top_p, top_class = ps.topk(1, dim=1)
      equals = top_class == labels.view(*top_class.shape)
      accuracy += torch.mean(equals.float())
      
  print(f"Subject {worker.id}.. "
    f"Test loss: {test_loss.get()/len(testloader):.3f}.. "
    f"Test accuracy: {accuracy.get()/len(testloader):.3f}")

Subject S5.. Test loss: 0.194.. Test accuracy: 0.967
Subject S2.. Test loss: 0.401.. Test accuracy: 0.890
Subject S3.. Test loss: 0.147.. Test accuracy: 1.000
Subject S4.. Test loss: 0.138.. Test accuracy: 0.933
Subject S10.. Test loss: 0.226.. Test accuracy: 0.866
Subject S11.. Test loss: 0.293.. Test accuracy: 0.915
Subject S8.. Test loss: 0.315.. Test accuracy: 0.843
Subject S1.. Test loss: 0.268.. Test accuracy: 0.925
Subject S6.. Test loss: 0.185.. Test accuracy: 1.000
Subject S7.. Test loss: 0.177.. Test accuracy: 0.935
Subject S9.. Test loss: 0.108.. Test accuracy: 1.000
Subject S13.. Test loss: 0.394.. Test accuracy: 0.883
Subject S12.. Test loss: 0.088.. Test accuracy: 1.000


## [Part 4] - Conclusions

Using PySyft, we were able to use disjoint private data to train a generalized model without having to compromise or move the data from it's original location. Even though the overall accuracy of our global model is lower than that seen on each individual's model, our model generates a good accuracy for all our subjects. This probably wouldn't happen with any remote model, as they are prone to overfitting on small and low diversity datasets. In a real life scenario, more subjects and more data would be collected, which would result in a more robust and generalized model, and therefore better overall accuracy.

In [20]:
print("Done with PySyft version " + sy.__version__)

Done with PySyft version 0.1.23a1
